In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt
from sqlalchemy.ext.declarative import declarative_base



In [2]:
import numpy as np
import pandas as pd
import datetime as dt

In [3]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func


In [4]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")
conn = engine.connect()


In [5]:
# Create a base class for automapping
Base = automap_base()

In [6]:
# reflect an existing database into a new model

Base.prepare(autoload_with=engine)

# reflect the tables
Base.classes.keys()



['station', 'measurement']

In [7]:
# Save references to each table

Station = Base.classes.station  
Measurement = Base.classes.measurement  

In [8]:
# Create our session (link) from Python to the DB
session = Session(engine)

Session

sqlalchemy.orm.session.Session

In [9]:
inspector = inspect(engine)
columns = inspector.get_columns('measurement')
for c in columns:
    print(c['name'], c["type"])

id INTEGER
station TEXT
date TEXT
prcp FLOAT
tobs FLOAT


In [10]:
inspector = inspect(engine)
columns = inspector.get_columns('station')
for c in columns:
    print(c['name'], c["type"])

id INTEGER
station TEXT
name TEXT
latitude FLOAT
longitude FLOAT
elevation FLOAT


In [11]:
# Find the most recent date in the data set.
recent_date = session.query(Measurement.date).order_by(Measurement.date.desc()).first()

recent_date

('2017-08-23',)

In [41]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 
# Calculate the date one year from the last date in data set.

twelve_months = dt.date(2017, 8, 23) - dt.timedelta(days=365)

twelve_months


datetime.date(2016, 8, 23)

In [45]:
# Perform a query to retrieve the data and precipitation scores
query_results = session.query(Measurement.date, Measurement.prcp).\
    filter(Measurement.date > twelve_months ).\
    filter(Measurement.prcp).all()

query_results

[('2016-08-24', 0.08),
 ('2016-08-25', 0.08),
 ('2016-08-28', 0.01),
 ('2016-08-31', 0.13),
 ('2016-09-04', 0.03),
 ('2016-09-07', 0.05),
 ('2016-09-09', 0.03),
 ('2016-09-11', 0.05),
 ('2016-09-13', 0.02),
 ('2016-09-14', 1.32),
 ('2016-09-15', 0.42),
 ('2016-09-16', 0.06),
 ('2016-09-17', 0.05),
 ('2016-09-22', 0.02),
 ('2016-09-26', 0.06),
 ('2016-09-27', 0.02),
 ('2016-10-17', 0.01),
 ('2016-10-21', 0.05),
 ('2016-10-22', 0.15),
 ('2016-10-23', 0.01),
 ('2016-10-25', 0.03),
 ('2016-10-30', 0.24),
 ('2016-10-31', 0.03),
 ('2016-11-08', 0.07),
 ('2016-11-19', 0.03),
 ('2016-11-20', 0.05),
 ('2016-11-21', 0.01),
 ('2016-11-22', 0.13),
 ('2016-11-23', 0.14),
 ('2016-11-24', 0.05),
 ('2016-11-25', 0.05),
 ('2016-11-26', 0.05),
 ('2016-11-28', 0.01),
 ('2016-11-30', 0.14),
 ('2016-12-01', 0.12),
 ('2016-12-02', 0.03),
 ('2016-12-04', 0.03),
 ('2016-12-05', 0.43),
 ('2016-12-06', 0.02),
 ('2016-12-08', 0.03),
 ('2016-12-09', 0.52),
 ('2016-12-10', 0.05),
 ('2016-12-11', 0.04),
 ('2016-12-

In [53]:
# Save the query results as a Pandas DataFrame. Explicitly set the column names

df = pd.DataFrame(query_results, columns=['Date', 'Precipitation'])

# df.rename(columns={'date': 'date', 'prcp': 'Precipitation'})
df.set_index('Date', inplace=True)

print(df)

            Precipitation
Date                     
2016-08-24           0.08
2016-08-25           0.08
2016-08-28           0.01
2016-08-31           0.13
2016-09-04           0.03
...                   ...
2017-08-17           0.13
2017-08-19           0.09
2017-08-21           0.56
2017-08-22           0.50
2017-08-23           0.45

[1142 rows x 1 columns]


In [55]:

# Sort the dataframe by date

df = df.sort_values(by='Date')

In [60]:
# Use Pandas Plotting with Matplotlib to plot the data
plt.figure(figsize=(14, 7)) 
df.plot(x='Date', y='Precipitation', kind='box')

plt.show()

KeyError: 'Date'

<Figure size 1400x700 with 0 Axes>

In [62]:
# Use Pandas to calculate the summary statistics for the precipitation data
summary_stats = df['Precipitation']. describe()

print(summary_stats)

count    1142.000000
mean        0.311357
std         0.576134
min         0.010000
25%         0.030000
50%         0.100000
75%         0.310000
max         6.700000
Name: Precipitation, dtype: float64


In [63]:
# Design a query to calculate the total number of stations in the dataset
station_count = session.query(func.count(Station.id)).scalar() 
print(station_count)

9


In [80]:
# Design a query to find the most active stations (i.e. which stations have the most rows?)
# List the stations and their counts in descending order.

most_active =session.query(Station.id, Station.station, func.count().label('activity_count')) \
    .group_by(Station.station) \
    .all()

# Print results
for station in most_active:
    print(f"Station ID: {station.id}, Station: {station.station}, Activity Count: {station.activity_count}")

Station ID: 8, Station: USC00511918, Activity Count: 1
Station ID: 2, Station: USC00513117, Activity Count: 1
Station ID: 3, Station: USC00514830, Activity Count: 1
Station ID: 9, Station: USC00516128, Activity Count: 1
Station ID: 4, Station: USC00517948, Activity Count: 1
Station ID: 5, Station: USC00518838, Activity Count: 1
Station ID: 7, Station: USC00519281, Activity Count: 1
Station ID: 1, Station: USC00519397, Activity Count: 1
Station ID: 6, Station: USC00519523, Activity Count: 1


In [90]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.
station_id = 8

temperature_stats = session.query(
    func.min(Measurement.tobs).label('min_temp'),
    func.max(Measurement.tobs).label('max_temp'),
    func.avg(Measurement.tobs).label('avg_temp')
).filter(Measurement.id == station_id).all()

for stats in temperature_stats:
    print(f"Station ID {station_id} - Min Temp: {stats.min_temp}, Max Temp: {stats.max_temp}, Avg Temp: {stats.avg_temp}")

Station ID 8 - Min Temp: 68.0, Max Temp: 68.0, Avg Temp: 68.0


In [91]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
active_recent = session.query(Measurement.tobs).\
    filter(Measurement.date > twelve_months ).\
    filter(Measurement.tobs).all()

active_recent

[(79.0,),
 (80.0,),
 (79.0,),
 (77.0,),
 (78.0,),
 (78.0,),
 (79.0,),
 (80.0,),
 (81.0,),
 (80.0,),
 (79.0,),
 (75.0,),
 (79.0,),
 (76.0,),
 (76.0,),
 (80.0,),
 (79.0,),
 (78.0,),
 (76.0,),
 (78.0,),
 (78.0,),
 (75.0,),
 (79.0,),
 (80.0,),
 (79.0,),
 (81.0,),
 (77.0,),
 (77.0,),
 (78.0,),
 (76.0,),
 (82.0,),
 (81.0,),
 (78.0,),
 (80.0,),
 (80.0,),
 (79.0,),
 (80.0,),
 (80.0,),
 (80.0,),
 (79.0,),
 (79.0,),
 (79.0,),
 (80.0,),
 (80.0,),
 (79.0,),
 (78.0,),
 (76.0,),
 (75.0,),
 (80.0,),
 (80.0,),
 (79.0,),
 (80.0,),
 (79.0,),
 (78.0,),
 (78.0,),
 (80.0,),
 (81.0,),
 (79.0,),
 (77.0,),
 (79.0,),
 (78.0,),
 (77.0,),
 (77.0,),
 (78.0,),
 (77.0,),
 (78.0,),
 (77.0,),
 (76.0,),
 (77.0,),
 (77.0,),
 (77.0,),
 (77.0,),
 (77.0,),
 (76.0,),
 (75.0,),
 (71.0,),
 (71.0,),
 (71.0,),
 (71.0,),
 (72.0,),
 (71.0,),
 (72.0,),
 (77.0,),
 (77.0,),
 (77.0,),
 (77.0,),
 (77.0,),
 (75.0,),
 (76.0,),
 (76.0,),
 (75.0,),
 (72.0,),
 (74.0,),
 (75.0,),
 (73.0,),
 (76.0,),
 (75.0,),
 (68.0,),
 (75.0,),
 (75.0,),


In [2]:
# Close Session
session.close()

NameError: name 'session' is not defined